In [1]:
import sys
sys.path.append("../../")
from taxcalc import *
import pandas as pd
import numpy as np
import copy

In [2]:
policy_cl = Policy()
behavior_cl = Behavior()
records_cl = Records("../../puf.csv")
calc_cl = Calculator(policy_cl, records_cl, behavior_cl)
for i in range(4):
    calc_cl.increment_year()
assert calc_cl.current_year == 2017
calc_cl.calc_all()
print "Done"

You loaded data for 2009.
Your data have been extrapolated to 2013.
Done


In [3]:
policy_t = Policy()
behavior_t = Behavior()
records_t = Records("../../puf.csv")
reform_t = {
    2017: {
        '_ID_cap': [100000]
    }
}
policy_t.implement_reform(reform_t)
calc_t = Calculator(policy_t, records_t, behavior_t)
for i in range(4):
    calc_t.increment_year()
assert calc_t.current_year == 2017
calc_t.calc_all()
print "Done"

You loaded data for 2009.
Your data have been extrapolated to 2013.
Done


In [4]:
policy_rb = Policy()
behavior_rb = Behavior()
records_rb = Records("../../puf.csv")
reform_rb = {
    2017: {
        '_II_rt1': [0.12],
        '_II_rt2': [0.12],
        '_II_rt3': [0.25],
        '_II_rt4': [0.25],
        '_II_rt5': [0.33],
        '_II_rt6': [0.33],
        '_II_rt7': [0.33],
        '_AMT_rt1': [0.0],
        '_AMT_rt2': [0.0],
        '_STD': [[12000, 24000, 12000, 18000, 24000, 12000, 1050]],
        '_STD_Aged': [[0,0,0,0,0,0],
                      [0,0,0,0,0,0],
                      [0,0,0,0,0,0],
                      [0,0,0,0,0,0]],
        '_II_em': [0,0,0,0],
        '_CTC_ps': [[75000, 150000, 75000, 75000, 75000, 75000]],
        '_DependentCredit_c': [500],
        '_ID_StateLocalTax_hc': [1.0],
        '_ID_Medical_hc': [1.0],
        '_ID_Casualty_hc': [1.0],
        '_ID_Miscellaneous_hc': [1.0],
        '_ID_RealEstate_hc': [1.0],
        '_ALD_Investment_ec_rt': [0.5],
        '_ALD_Investment_ec_base_all': [False],
        '_CG_nodiff': [True],
        '_NIIT_rt': [0.0],
        '_PT_rt1': [0.12],
        '_PT_rt2': [0.12],
        '_PT_rt3': [0.25],
        '_PT_rt4': [0.25],
        '_PT_rt5': [0.25],
        '_PT_rt6': [0.25],
        '_PT_rt7': [0.25]
    }
}
policy_rb.implement_reform(reform_rb)
calc_rb = Calculator(policy_rb, records_rb, behavior_rb)
for i in range(4):
    calc_rb.increment_year()
assert calc_rb.current_year == 2017
calc_rb.calc_all()
print "Done"

You loaded data for 2009.
Your data have been extrapolated to 2013.
Done


In [5]:
policy_rbmini = Policy()
behavior_rbmini = Behavior()
records_rbmini = Records("../../puf.csv")
reform_rbmini = {
    2017: {
        '_II_rt1': [0.10],
        '_II_rt2': [0.15],
        '_II_rt3': [0.25],
        '_II_rt4': [0.28],
        '_II_rt5': [0.33],
        '_II_rt6': [0.33],
        '_II_rt7': [0.33],
        '_NIIT_rt': [0.0]
    }
}
policy_rbmini.implement_reform(reform_rbmini)
calc_rbmini = Calculator(policy_rbmini, records_rbmini, behavior_rbmini)
for i in range(4):
    calc_rbmini.increment_year()
assert calc_rbmini.current_year == 2017
calc_rbmini.calc_all()
print "Done"

You loaded data for 2009.
Your data have been extrapolated to 2013.
Done


In [6]:
def getRevenueImpact(calc_base, calc_reform):
    comb_base = calc_base.records._combined * calc_base.records.s006
    comb_reform = calc_reform.records._combined * calc_reform.records.s006
    diff = (comb_reform - comb_base).sum() / 10**9
    return diff

print getRevenueImpact(calc_cl, calc_t)
print getRevenueImpact(calc_cl, calc_rb)
print getRevenueImpact(calc_cl, calc_rbmini)

21.2874809545
-264.097529071
-55.3041014605


In [8]:
ided_ind = [i for i in range(219814) if calc_cl.records.c04470[i] > calc_cl.records._standard[i]]
highided_bin = np.where((((calc_cl.records.MARS == 2) & (calc_cl.records.c04470 >= 200000)) |
                         ((calc_cl.records.MARS != 2) & (calc_cl.records.c04470 >= 100000))) == True, 1, 0)
highided_ind = [i for i in ided_ind if highided_bin[i] == 1]

def num_ided(calc):
    num = np.where(calc.records.c04470 > calc.records._standard, calc.records.s006, 0).sum()
    return num
def amt_ided(calc):
    amt = np.where(calc.records.c04470 > calc.records._standard, calc.records.c04470 * calc.records.s006, 0).sum()
    return amt
def amt_highided(calc):
    amth = sum([calc.records.c04470[i] * calc.records.s006[i] for i in highided_ind])
    return amth
def mtr_highided(calc):
    mtr_all = calc.mtr('e00200p')[1]
    wmtr = (sum([mtr_all[i] * calc.records.e00200[i] * calc.records.s006[i] for i in highided_ind]) /
            sum([calc.records.e00200[i] * calc.records.s006[i] for i in highided_ind]))
    return wmtr
def avgRate_highided(calc):
    totaltax = [calc.records._combined[i] * calc.records.s006[i] for i in highided_ind]
    totalagi = [calc.records.c00100[i] * calc.records.s006[i] for i in highided_ind]
    avg = sum(totaltax) / sum(totalagi)
    return avg

In [9]:
print num_ided(calc_cl) / 1000.
print sum([calc_cl.records.s006[i] for i in highided_ind]) / 1000.
print amt_ided(calc_cl) / 10**9
print amt_highided(calc_cl) / 10**9
print mtr_highided(calc_cl)
print avgRate_highided(calc_cl)

45766.0477169
329.11759954
1363.58819699
119.244022136
0.384119126539
0.314768324568


In [10]:
print num_ided(calc_t) / 1000.
print amt_ided(calc_t) / 10**9
print amt_highided(calc_t) / 10**9
print mtr_highided(calc_t)
print avgRate_highided(calc_t)

45740.0403343
1288.90366445
44.5583631128
0.378012311686
0.34514289187


In [10]:
print num_ided(calc_rb)
print amt_ided(calc_rb) / 10**9
print amt_highided(calc_rb) / 10**9
print mtr_highided(calc_rb)
print avgRate_highided(calc_rb)

8366820.10287
254.063370028
28.0035198135
0.312275887651


In [11]:
print sum([calc_cl.records.e18400[i] * calc_cl.records.s006[i] for i in highided_ind]) / 10**9
print sum([calc_cl.records.e19200[i] * calc_cl.records.s006[i] for i in highided_ind]) / 10**9
print sum([(calc_cl.records.e19800[i] + calc_cl.records.e20100[i]) * calc_cl.records.s006[i] for i in highided_ind]) / 10**9


59.3345094954
16.2100075905
37.1882237575


In [4]:
print 46 / sum(calc_cl.records.s006) * 10**6

0.26682132891


In [7]:
def static_path_change(calcx, calcy):
    static_path = [0] * 10
    calc1 = copy.deepcopy(calcx)
    calc2 = copy.deepcopy(calcy)
    for i in range(10):
        calc1.calc_all()
        calc2.calc_all()
        calc1_combined = (calc1.records._combined * calc1.records.s006)
        calc2_combined = (calc2.records._combined * calc2.records.s006)
        static_path[i] = (calc2_combined - calc1_combined).sum() / 10**9
        if calc1.current_year < 2026:
            calc1.increment_year()
            calc2.increment_year()
    return static_path

In [8]:
print sum(static_path_change(calc_cl, calc_t))

273.785442079


In [9]:
print sum(static_path_change(calc_cl, calc_rbmini))

-662.206342845


In [16]:
print sum([calc_cl.records.c00100[i] * calc_cl.records.s006[i] for i in highided_ind]) / sum([calc_cl.records.s006[i] for i in highided_ind])
print np.percentile([calc_cl.records.c00100[i] for i in highided_ind], 50)
print sum([calc_cl.records.s006[i] for i in highided_ind if calc_cl.records.c00100[i] > 434700]) / sum([calc_cl.records.s006[i] for i in highided_ind])

2129470.29003
2414661.55061
0.649525661036


In [23]:
print np.percentile([calc_cl.records.c04470[i] for i in range(219814) if (calc_cl.records.MARS[i] == 2 and calc_cl.records.c04470[i] > calc_cl.records._standard[i])], 50)

43375.6987034


In [26]:
def extract_percentile(measure1, wgt, pctl):
    # Extracts the observation at the low end of the given percentile
    # Percentile must be in decimal form
    assert pctl >=0 and pctl <=1
    total = sum(wgt)
    paired = zip(measure1, wgt)
    paired.sort()
    paired_sorted = zip(*paired)
    measure1_s = paired_sorted[0]
    wgt_s = paired_sorted[1]
    wgt_sum = 0
    i = 0
    while wgt_sum < total * pctl:
        wgt_sum += wgt_s[i]
        i += 1
    pctl_entry = measure1_s[i - 1]
    return pctl_entry
wgt1 = [calc_cl.records.s006[i] for i in range(219814) if (calc_cl.records.MARS[i] == 1 and calc_cl.records.c04470[i] > calc_cl.records._standard[i])]
ided1 = [calc_cl.records.c04470[i] for i in range(219814) if (calc_cl.records.MARS[i] ==1  and calc_cl.records.c04470[i] > calc_cl.records._standard[i])]
print extract_percentile(ided1, wgt1, 0.5)

15375.8406758
